## Download and check rdf2vec

In [1]:
VECTOR_SIZE=100

## Download Dataset


In [2]:
# download .nt dataset from my drive
! wget -q -nc --no-check-certificate 'https://docs.google.com/uc?export=download&id=1pBnn8bjI2VkVvBR33DnvpeyocfDhMCFA' -O fb15k-237_nt.zip

! unzip -n fb15k-237_nt.zip

Archive:  fb15k-237_nt.zip


In [3]:
import numpy as np
import gc

def get_entities(graphs):
    # get subjects and objects
    entities = []
    
    for g in graphs:
        entities = entities + list(g.subjects(unique=True)) + list(g.objects(unique=True))

    # pythons stupid version of nub
    entities = list(dict.fromkeys(entities))
    return entities

def get_all_corrupted_triples_fast(triple,entities,position = 'object'):
    # not faster ...

    s,p,o = triple

    object_augmented = [(x,y,z) for  (x,y), z in itertools.product([triple[0:2]],entities)]
    subject_augmented =[(x,y,z) for  x, (y,z) in itertools.product(entities,[triple[1:3]])]
    
    
    return itertools.chain(object_augmented , subject_augmented)

def get_all_corrupted_triples(triple,entities):
    #too slow ....
    
    s,p,o = triple
    subject_corrupted = [(s_corr,p,o) for s_corr in entities if s_corr != s]
    object_corrupted = [(s,p,o_corr)   for o_corr in entities if o_corr != o]

    return subject_corrupted + object_corrupted


    

def choose_many_multiple(arrs,n):
    l = len(arrs[0])
    for a in arrs:
        assert len(a) == l, 'Arres not of same length ! :('
        
    
    ix = np.random.choice(range(len(a)),n)
    
    return [np.array(a)[ix] for a in arrs]
    
def choose_many(a,n):
    ix = np.random.choice(range(len(a)),n)
    return np.array(a)[ix]
    
def choose(a):

    L = len(a)

    i = np.random.randint(0,L)

    return a[i]

def get_random_corrupted_triple(triple,entities, corrupt='object'):
    """
    corrupt = one of 'subject', 'object', 'both'
    
    return corrupted triple with random entity
    """

    s,p,o = triple
    
    # set up as the same
    s_corr = s
    o_corr = o
    
    if corrupt == 'subject':  
        # corrupt only the subject
        while s_corr == s:
            s_corr = choose(entities)  
    elif corrupt == 'object':
        # corrupt only the object
        while o_corr == o:
            o_corr = choose(entities)  
    elif corrupt == 'random':
        # corrupt one or both randomly
        ch = np.random.randint(3)
        
        if ch == 0:
            while s_corr == s:
                s_corr = choose(entities)  
        if ch == 1 :
            while o_corr == o:
                o_corr = choose(entities)  
        if ch == 2:
            while s_corr == s or o_corr == o:
                s_corr = choose(entities)  
                o_corr = choose(entities) 
    else:
        while s_corr == s or o_corr == o:
            s_corr = choose(entities)  
            o_corr = choose(entities) 
            
    
    return (s_corr,p,o_corr)
    
def merge_historires(history_list):
    h = {}
    for key in history_list[0].history.keys():
        h[key] = [h.history[key][0] for h in histories]
    return h    


def clean_graph(graph,wv):
    """
    clean graph such that all triples have word vectors present in wv
    
    """
    no_removed = 0 
    for t in graph:
        s,p,o = t
        if not str(s) in wv.key_to_index.keys() or not str(p) in wv.key_to_index.keys() or not str(o) in wv.key_to_index.keys():
            graph.remove(t)
            no_removed+=1
    return no_removed
    
    
def get_vectors_fast(triples,entity_vec_mapping,vector_size=VECTOR_SIZE):
    # ~20-30% faster
    X = np.array(triples)
    X = word_vectors[X.flatten()].reshape(len(triples),vector_size*3)
    
    return X    

def get_vectors(triples,entity_vec_mapping,vector_size=200):
    X = np.array(triples)
    X = [(entity_vec_mapping(x[0]), entity_vec_mapping(x[1]),entity_vec_mapping(x[2])) for x in X]
    X = [np.concatenate(x) for x in X]
    X = np.vstack(X).astype(np.float64)
    
    return X

def get_1_1_dataset(graph, entities,entity_vec_mapping,corrupt='random'):
    
    original_triple_len = len(graph)
    # get triples
    X = list(graph)
    no_t = len(X)
    

    
    corrupted_triples = [get_random_corrupted_triple(x,entities,corrupt=corrupt) for x in X]
    X = X + corrupted_triples
    
    

    # convert uris to strings
    
    X = get_vectors_fast(X,entity_vec_mapping)
    
    # stack them

    Y = np.concatenate((np.ones(no_t),np.zeros(no_t))).astype(np.uint8)
    
    return X, Y

def test_sklearn_model(model,X,Y,x_test,y_test,subset=10000):
    

  
    
    ix = np.random.choice(range(len(X)),size=subset)
    
    scaler = preprocessing.StandardScaler().fit(X)
    
    X_scaled = scaler.transform(X[ix])
    model.fit(X_scaled,Y[ix])

    print(f'train_score ={model.score(scaler.transform(X),Y)}')    
    print(f'test_score ={model.score(scaler.transform(x_test),y_test)}')

def scale_and_predict(model,x):
    x = preprocessing.StandardScaler().fit_transform(x)
    return model.predict(x)

## Parse Graph

In [4]:
from rdflib import Graph, URIRef
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
import networkx as nx



g_train = Graph()
g_val = Graph()
g_test = Graph()

g_train = g_train.parse('FB15k-237/train.nt', format='nt')
g_val   = g_val.parse('FB15k-237/valid.nt', format='nt')
g_test  = g_test.parse('FB15k-237/test.nt', format='nt')


In [5]:
from gensim.models import Word2Vec


word_vectors = Word2Vec.load('walks/model').wv

In [6]:
def map_keyed_vectors(word_vectors, iterable):
    """
    for some reason faster than native call :O
    """
    return np.array(list(word_vectors.get_vector(x) for x in iterable))

In [7]:
# pytorch model
import torchmetrics
import torch
from torch import nn 


class ClassifierSimple(torch.nn.Module):
    def __init__(self,input_dim=300,hidden_size=64):
        super(ClassifierSimple, self).__init__()
        
        
        self.layers = nn.Sequential(
                # flatten input if necessary
                nn.Flatten(),
                nn.Linear(input_dim,hidden_size),
                nn.ReLU(),
                nn.Linear(hidden_size,1)
        )
        
        self.output_activation = nn.Sigmoid()
                
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
 
        
    
    def forward(self,x):        
        
        return self.layers(x)
    def predict(self,x):
        x.to(self.device)
        
        return self.output_activation(self.layers(x))
    def predict_numpy(self,x):
        x = torch.tensor(x)
        x.to(self.device)
        return self.output_activation(self.layers(x)).detach().cpu().numpy()
        
    


2022-11-12 09:05:43.548095: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-12 09:05:43.548115: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [8]:
from pathlib import Path
import pandas as pd
model = ClassifierSimple()    

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if Path('rdf2vecClassfier.pth').is_file():
    print('found trained model! Loading :)')
    model.load_state_dict(torch.load('rdf2vecClassfier.pth'))
    history = pd.read_csv('log.csv')
    model = model.to(device)
else:
    print('model not found. Train it with ''train_rdf2vec_classifier.ipynb')

found trained model! Loading :)


In [9]:
import time
from tqdm import tqdm
from collections import defaultdict
from itertools import chain
import pickle


In [10]:
def fast_concat(se,pe,oe):
    
    assert se.shape == pe.shape, "Error! fast_concat with differing shapes"
    assert se.shape == oe.shape, "Error! fast_concat with differing shapes"
    
    
    x = np.empty((se.shape[0],se.shape[1]*3),dtype=np.float32)
    x[:,0:100] =se
    x[:,100:200] = pe
    x[:,200:] = oe
    
    return x

In [11]:
def get_all_s_o_tuples(entities):
    return [(s,o) for s in entities for o in entities] + [(o,s) for s in entities for o in entities]
    
    

In [12]:
def test_if_row_in_array(row,arr):
    return np.any(np.sum(arr == row,axis=1) == 3)

In [13]:
def get_filter(triples,graphs):
    # Too slow ... parallelize?! What else? Faster lookup? How?
    graphs = [np.array(g) for g in graphs]

    graphs = np.concatenate(graphs,axis=0)
    
    known_ix = []
    
    for i,tp in enumerate(triples):
        if test_if_row_in_array(tp,graphs):
            known_ix.append(i)
    return known_ix
    
    

In [14]:
def map_keyed_vectors(word_vectors, iterable):
    """
    for some reason faster than native call :O
    """
    return np.array(list(word_vectors.get_vector(x) for x in iterable))

In [15]:
import time
from sklearn.utils.extmath import cartesian

def evaluate_link_pred_fast(score_f,graph,entity_vec_mapping,entities,vector_size = 100, max_triples=100, plot = False, filter_by=None,verbose = True):
    
    
    
    stats = {'preprocessing_time' : -1,
        'embeddings_time': [],
        'rank_time': [],
        'find_rank_time':[],
        'ranks':[]
        }
    
    start_timer = time.perf_counter()
    predicates = np.array(list(set(graph.predicates())))
    
    graph = np.array(graph)
    
    
    print(f"evaluate LP on graph with {len(graph)} triples, {len(entities)} entities and {len(predicates)} predicates!")
    
    print(f"Starting preprocessing")
    embeddings_scores = defaultdict(lambda: defaultdict(lambda :None))
    
    entity_array = np.array(entities)
    entity_mapping = entity_vec_mapping(np.array(entity_array))
    
    no_entitites= len(entities)
    ix = list(range(no_entitites))
    
    s_o_combinations = cartesian((ix,ix))   

    subjects_ix = s_o_combinations[:,0] # sorted(s_o_combinations[:,0])
    subject_embeddings = entity_mapping[subjects_ix]
                                 

    
    objects_ix = s_o_combinations[:,1]  #sorted(s_o_combinations[:,1])
    object_embeddings = entity_mapping[objects_ix]
    
                           
    no_triples_per_predicate=len(subjects_ix)
    
    
    
    preprocessing_timer = time.perf_counter()
    
    stats['preprocessing_time'] = preprocessing_timer - start_timer
    
    print(f"Finished preprocessing")
    
    # del is very slow :(
    # del s_o_combinations
    s_o_combinations = None
    #gc.collect()
    
    
    for p in tqdm(predicates):
        predicate_start_timer = time.perf_counter()
        
        
             
        predicate_embedding = entity_vec_mapping([p])
        predicate_column = np.repeat(predicate_embedding,no_triples_per_predicate).reshape(no_triples_per_predicate,vector_size)
        
        #return subject_embeddings,predicate_column,object_embeddings

        triple_embeddings = fast_concat(subject_embeddings,predicate_column,object_embeddings)
        
        
        

        predicate_embeddings_timer = time.perf_counter()
        
        #del predicate_column
        predicate_column = None

        

        scores = np.squeeze(score_f(triple_embeddings))
        
        #del triple_embeddings
        triple_embeddings = None
        #gc.collect()
        
        
        
        sorted_ix = np.flip(np.argsort(scores))     
        
        subjects = entity_array[subjects_ix] 
        objects = entity_array[objects_ix]
    
        scored_triples = np.stack([subjects,np.repeat(p,no_triples_per_predicate),objects]).T   
        scored_triples = scored_triples[sorted_ix]
        
        
        predicate_rank_timer = time.perf_counter()
        
        ranks = []
    

        predicate_subgraph = graph[graph[:,1] == p]
    
    
        
        for triple in predicate_subgraph:
            try:
                rank = np.where(np.sum(scored_triples ==triple,axis=1) == 3)[0][0]
                print(rank)
                ranks.append(rank)
            except:
                print(triple)
                print(np.where(np.sum(scored_triples ==triple,axis=1)))
                print('unknown entity or relation!')
                
        
        
        predicate_find_rank = time.perf_counter()
        
            
        stats['embeddings_time'].append(predicate_embeddings_timer-predicate_start_timer)
        stats['rank_time'].append(predicate_rank_timer - predicate_embeddings_timer)
        stats['find_rank_time'].append(predicate_find_rank - predicate_rank_timer)
        stats['ranks'].extend(ranks)
        
       
        

    return  stats

In [16]:
from rdflib import Graph, URIRef
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
import networkx as nx



g_train = Graph()
g_val = Graph()
g_test = Graph()

g_train = g_train.parse('FB15k-237/train_1000.nt', format='nt')
g_val   = g_val.parse('FB15k-237/valid_1000.nt', format='nt')
g_test  = g_test.parse('FB15k-237/test_1000.nt', format='nt')


# clean graphs 
# number of triples removed should be low, a few hundred
print(f"removed {clean_graph(g_train,word_vectors)} triples from training set")
print(f"removed {clean_graph(g_val,word_vectors)} triples from validation set")
print(f"removed {clean_graph(g_test,word_vectors)} triples from test set")

entities = get_entities((g_train,g_val,g_test))

removed 2 triples from training set
removed 2 triples from validation set
removed 0 triples from test set


In [17]:
def compute_rank(scores,ix,mask=None):
    if mask == None:    
        optimistic_rank =(scores > scores[ix]).sum()+1
        pessimistic_rank = (scores >= scores[ix]).sum()

        
    else:
        
        optimistic_rank = ((scores > scores[ix]).index_fill(0,mask,False)).sum()+1
        pessimistic_rank = ((scores >= scores[ix]).index_fill(0,mask,False)).sum()
        
    rank = (optimistic_rank+pessimistic_rank)*0.5
        
    return rank

def parse_rdflib_to_torch(graph):
    entities = get_entities([graph])
    
    entities=np.array(entities)
    entity_vecs= torch.tensor(word_vectors[np.array(entities)])
    entities = dict(zip(entities,range(len(entities))))
    
    predicates = np.array(list(set(graph.predicates())))
    predicate_vecs = torch.tensor(word_vectors[predicates])
    predicates = dict(zip(predicates,range(len(predicates))))
    
    edges = []
    predicate_ix = []
    for s,p,o in np.array(graph):
        try:
            edge = (entities[s],entities[o])
            edges.append(edge)
            predicate_ix.append(predicates[p])
        except:
            print(f"Unknown entities encountered! ({s},{o})")

    edges = torch.tensor(edges)
    predicate_ix = torch.tensor(predicate_ix)
    
    return edges, predicate_ix, entities, predicates,entity_vecs,predicate_vecs

def get_comb_ix(edge,no_entities):
    return edge[0]*no_entities+edge[1]

def create_mask(length,ix,reverse=False):
    if not reverse:
        mask = torch.ones(length)
        mask = mask.index_fill(0,ix,0)
    else:
        mask = torch.zeros(length)
        mask = mask.index_fill(0,ix,1)
    
    
    return mask

In [21]:
from timer import PerfTimer
from torch.utils.data import DataLoader

def eval_ranks(graph,filtered = True, batchsize=None, vecsize=100):
# get data from graph

    perfTimer = PerfTimer()
    
    perfTimer.start()
    
    edges, predicate_ix, entity_ix_mapping, predicate_ix_mapping,entity_vecs,predicate_vecs = parse_rdflib_to_torch(graph)
    
    ranks = []
    ix_predicate_mapping = {k:v for (v,k) in predicate_ix_mapping.items()}
    ix_entity_mapping = {k:v for (v,k) in entity_ix_mapping.items()}
    
    no_entities  = len(entity_ix_mapping.keys())
    
    perfTimer.track('preprocessing')    
    
    for pred_ix in tqdm(ix_predicate_mapping.keys()):
        

        current_edges = edges[predicate_ix == pred_ix]
        number_of_real_edges = len(current_edges)
        
        perfTimer.track('subgraph')
        # could also try torch.cross and torch.combinations if needed
        # all possible s_o_combinations
        s_o_combs = torch.tensor(cartesian((range(no_entities),range(no_entities))))
    


        edge_ix = torch.tensor([get_comb_ix(x,no_entities) for x in current_edges])
        perfTimer.track('edges')
        
        if batchsize:
            dl = DataLoader(s_o_combs, batch_size=batchsize, shuffle=False)
            perfTimer.track('dl')
            predicate_embedding = predicate_vecs[pred_ix]
            #use expand as memory of rows is shared. This may cause bugs ... investigate!
            predicate_embedding = predicate_embedding.reshape(1,vecsize).expand(batchsize,vecsize)
            perfTimer.track('expand_pediacte')
            
            scores = []

            for batch in dl:
                                #print(s_o_combs[:,0].type())
                #print(batch[:,0].type())
                subject_embeddings = entity_vecs[batch[:,0]]
                object_embeddings = entity_vecs[batch[:,1]]
                perfTimer.track('collect_embeddings_batch')
                
                if len(batch) != batchsize:
                    predicate_embedding = predicate_vecs[pred_ix]
                    predicate_embedding = predicate_embedding.reshape(1,vecsize).expand(len(batch),vecsize)
                
                to_score_embeddings = torch.hstack((subject_embeddings,predicate_embedding,object_embeddings))
                perfTimer.track('stack_batch')
                
                batch_scores = model(to_score_embeddings)
                perfTimer.track('predict_batch')
                scores.append(batch_scores)
                
            scores = torch.vstack(scores).squeeze()
            perfTimer.track('stack_all')
            
        else:
            
            subject_embeddings = entity_vecs[s_o_combs[:,0]]
            object_embeddings = entity_vecs[s_o_combs[:,1]]

            predicate_embedding = predicate_vecs[pred_ix]
            #use expand as memory of rows is shared. This may cause bugs ... investigate!
            predicate_embedding = predicate_embedding.reshape(1,100).expand(len(subject_embeddings),100)
            
            perfTimer.track('collect_embeddings')

            to_score_embeddings = torch.hstack((subject_embeddings,predicate_embedding,object_embeddings))
            perfTimer.track('stack_embeddings')

            scores = model(to_score_embeddings).squeeze()
            perfTimer.track('score_embeddings')
        #sorted_ix = scor

        

        
        for ix in edge_ix:
            ranks.append(compute_rank(scores,ix,edge_ix))
        perfTimer.track('rank_embeddings')
        
    return torch.tensor(ranks), perfTimer

    

In [38]:
import gc
gc.collect()

63985

In [ ]:
with torch.no_grad():
    ranks, pt = eval_ranks(g_train,)
    print(ranks.mean())
    print((1/ranks).mean())
gc.collect()
#pt.stats()

 65%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                                               | 102/156 [03:08<01:41,  1.88s/it]

In [34]:
d

defaultdict(list,
            {'preprocessing': [0.04247043799978201],
             'subgraph': [0.0026427510001667542,
              0.0013824559996464814,
              0.000854070000059437,
              0.0009705590000521624,
              0.000983723999979702,
              0.0012053820000801352,
              0.0008695289998286171,
              0.0009200029999192338,
              0.0008950860001277761,
              0.0008502919999955338,
              0.0008321279997289821,
              0.0015965679999681015,
              0.0008076009999058442,
              0.0008699690001776617,
              0.0008807889998934115,
              0.00106397499985178,
              0.0009528949999548786,
              0.0013091370001347968,
              0.0013053389998276543,
              0.0014187130000209436,
              0.0011429230003159319,
              0.0008610220002083224,
              0.0009702070001367247,
              0.0013945770001555502,
              0.00079707200029588

In [ ]:
{key: sum(value) for key,value in pt.stats().items()}

In [ ]:
with torch.no_grad():
    ranks = eval_ranks(g_train,False)
    print(ranks.mean())
    print((1/ranks).mean())
gc.collect()